In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# Generate RSA key pair
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# Save private key
with open("private.pem", "wb") as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save public key
with open("public.pem", "wb") as f:
    f.write(public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

print("Private & public keys generated!")


Private & public keys generated!


In [ ]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
import datetime

import tomllib as tl

with open("config.toml", "rb") as f:
    config = tl.load(f)['CA']

# Generate RSA Key Pair
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

# Create a Self-Signed X.509 Certificate
subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COMMON_NAME, config['COMMON_NAME']),
    x509.NameAttribute(NameOID.COUNTRY_NAME, config['COUNTRY_NAME']),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, config['ORGANIZATION_NAME']),
    x509.NameAttribute(NameOID.ORGANIZATION_IDENTIFIER, config['ORGANIZATION_IDENTIFIER']),
    x509.NameAttribute(NameOID.SERIAL_NUMBER, config['SERIAL_NUMBER']),
])


cert = x509.CertificateBuilder().subject_name(subject)\
    .issuer_name(issuer)\
    .public_key(public_key)\
    .serial_number(x509.random_serial_number())\
    .not_valid_before(datetime.datetime.now(datetime.UTC))\
    .not_valid_after(datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=365))\
    .sign(private_key, hashes.SHA256())

# Save Private Key
with open("private.pem", "wb") as f:
    f.write(private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    ))

# Save Public Certificate (X.509)
with open("certificate.pem", "wb") as f:
    f.write(cert.public_bytes(serialization.Encoding.PEM))

print("✅ X.509 Certificate & RSA Key Pair Generated!")


✅ X.509 Certificate & RSA Key Pair Generated!


In [3]:
import jwt
import json

# Load JSON invoice
with open("invoice.json", "r") as f:
    invoice_data = json.load(f)

# Load private key
with open("private.pem", "rb") as f:
    private_key = f.read()

# Create JWS signature
signed_invoice = jwt.encode(
    invoice_data,
    private_key,
    algorithm="RS256",
    headers={"alg": "RS256", "typ": "JWT"}
)

# Save signed JSON invoice
with open("signed_invoice.json", "w") as f:
    json.dump({"signed_invoice": signed_invoice}, f, indent=4)

print("Invoice successfully signed!")


Invoice successfully signed!


In [4]:
# Load public key
with open("public.pem", "rb") as f:
    public_key = f.read()

# Verify signature
decoded_invoice = jwt.decode(signed_invoice, public_key, algorithms=["RS256"])
print("Verified Invoice:", decoded_invoice)


Verified Invoice: {'invoice_id': 'INV-001', 'issue_date': '2025-03-21', 'supplier': {'name': 'My Company Sdn Bhd', 'id': '1234567890'}, 'customer': {'name': 'Customer Sdn Bhd', 'id': '0987654321'}, 'total_amount': 100.0, 'currency': 'MYR'}


In [12]:
import json
import base64
import jwt  # PyJWT
from cryptography.hazmat.primitives import serialization

def load_private_key():
    """Load private key from file."""
    with open("private.pem", "rb") as f:
        # return f.read()
        return serialization.load_pem_private_key(f.read(), password=None)

def preprocess_json_invoice(input_file, output_file):
    """Remove UBLExtensions & Signature, then minify JSON."""
    with open(input_file, "r") as f:
        invoice_data = json.load(f)
    
    # Remove unnecessary sections
    invoice_data.pop("UBLExtensions", None)
    invoice_data.pop("Signature", None)
    
    # Minify JSON (remove spaces/newlines)
    minified_json = json.dumps(invoice_data, separators=(",", ":"))
    
    with open(output_file, "w") as f:
        f.write(minified_json)
    
    return minified_json

def preprocess_dict_invoice(invoice_data):
    """Remove UBLExtensions & Signature from dict."""
    # Remove unnecessary sections
    invoice_data.pop("UBLExtensions", None)
    invoice_data.pop("Signature", None)
    return invoice_data

def sign_json_invoice(minified_json, private_key):
    """Sign JSON invoice using X.509 and JWS (RS256)."""
    return jwt.encode(
        json.loads(minified_json),  # Payload
        private_key,
        algorithm="RS256",
        headers={"alg": "RS256", "typ": "JWT"}
    )
    
def sign_dict_invoice(invoice_data, private_key):
    """Sign JSON invoice using X.509 and JWS (RS256)."""
    return jwt.encode(
        preprocess_dict_invoice(invoice_data),  # Payload
        private_key,
        algorithm="RS256",
        headers={"alg": "RS256", "typ": "JWT"}
    )

def encode_signed_invoice(signed_invoice):
    """Base64 encode the signed JSON invoice."""
    return base64.b64encode(signed_invoice.encode()).decode()

def main():
    private_key = load_private_key()
    minified_json = preprocess_json_invoice("invoice.json", "minified_invoice.json")
    signed_invoice = sign_json_invoice(minified_json, private_key)
    encoded_invoice = encode_signed_invoice(signed_invoice)
    
    with open("signed_invoice.json", "w") as f:
        json.dump({"signed_invoice": encoded_invoice}, f, indent=4)
    
    print("✔ Invoice signed and encoded successfully!")

if __name__ == "__main__":
    main()

✔ Invoice signed and encoded successfully!


generate X.509 certificate using cryptography